In [ ]:
from flask import render_template, request
from app import app
from models import Historical, Forecast, Station, Current
import datetime

import plotly
import plotly.graph_objs as go 

import pandas as pd
import numpy as np 
import json

In [ ]:
print(datetime.datetime.now())
print(datetime.date.today())

In [2]:
@app.route('/')
def index():   

    line = create_plot()
    return render_template("index.html", plot=line)    
    
def create_plot():
    N = 40
    now = datetime.datetime.now()
    x00 = [x.id for x in Historical.query.filter(Historical.id >= now - datetime.timedelta(days=7)).order_by(Historical.id)]
    y00 = [x.drybulb for x in Historical.query.filter(Historical.id >= now - datetime.timedelta(days=7)).order_by(Historical.id)]

    x0 = [x.id for x in Current.query.all()]
    y0 = [x.drybulb for x in Current.query.all()]

    # latest forecast (retrieved hourly)
    # just select the latest 
    
    # x1 = 

    # y1 =   
        
    # retrieved on roughly this hour, 1d ago
    # datetime greater than now minus 1d; less than five minutes after now minus 1d
    
    x2 = [x.id for x in Forecast.query.filter(Forecast.retrieval_time > now - datetime.timedelta(days=1),
        Forecast.retrieval_time < now - datetime.timedelta(minutes=1435)).order_by(Forecast.id)]

    y2 = [x.id for x in Forecast.query.filter(Forecast.retrieval_time > now - datetime.timedelta(days=1),
        Forecast.retrieval_time < now - datetime.timedelta(minutes=1435)).order_by(Forecast.id)]

    # retrieved on roughly this hour, 2d ago
    # datetime greater than now minus 2d; less than five minutes after now minus 2d
    
    x3 = [x.id for x in Forecast.query.filter(Forecast.retrieval_time > now - datetime.timedelta(days=2),
        Forecast.retrieval_time < now - datetime.timedelta(minutes=2875)).order_by(Forecast.id)]

    y3 = [x.id for x in Forecast.query.filter(Forecast.retrieval_time > now - datetime.timedelta(days=2),
        Forecast.retrieval_time < now - datetime.timedelta(minutes=2875)).order_by(Forecast.id)]

    historicals = go.Line(
            x=x00,
            y=y00,
            name='historicals'
            )

    actuals = go.Line(
            x=x0,
            y=y0,
            name='actuals'
            )
    

    forecasts_1d = go.Line(
            x=x2,
            y=y2,
            name='forecasts_1d'
            )

    forecasts_2d = go.Line(
            x=x3,
            y=y3,
            name='forecasts_2d'
            )

    data = [historicals, actuals, forecasts_1d, forecasts_2d]
    graphJSON = json.dumps(data, cls=plotly.utils.PlotlyJSONEncoder)

    return graphJSON

In [3]:
create_plot()

/Users/home/anaconda/envs/weather/lib/python3.7/site-packages/plotly/graph_objs/_deprecations.py:372: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




'[{"x": ["2019-06-13 17:00:00", "2019-06-13 18:00:00", "2019-06-13 19:00:00", "2019-06-13 20:00:00", "2019-06-13 21:00:00", "2019-06-13 22:00:00", "2019-06-13 23:00:00", "2019-06-14", "2019-06-14 01:00:00", "2019-06-14 02:00:00", "2019-06-14 03:00:00", "2019-06-14 04:00:00", "2019-06-14 05:00:00", "2019-06-14 06:00:00", "2019-06-14 07:00:00", "2019-06-14 08:00:00", "2019-06-14 09:00:00", "2019-06-14 10:00:00", "2019-06-14 11:00:00", "2019-06-14 12:00:00", "2019-06-14 13:00:00", "2019-06-14 14:00:00", "2019-06-14 15:00:00", "2019-06-14 16:00:00", "2019-06-14 17:00:00", "2019-06-14 18:00:00", "2019-06-14 19:00:00", "2019-06-14 20:00:00", "2019-06-14 21:00:00", "2019-06-14 22:00:00", "2019-06-14 23:00:00", "2019-06-15", "2019-06-15 01:00:00", "2019-06-15 02:00:00", "2019-06-15 03:00:00", "2019-06-15 04:00:00", "2019-06-15 05:00:00", "2019-06-15 06:00:00", "2019-06-15 07:00:00", "2019-06-15 08:00:00", "2019-06-15 09:00:00", "2019-06-15 10:00:00", "2019-06-15 11:00:00", "2019-06-15 12:00:00